In [1]:
import pandas as pd
import numpy as np

In [2]:
수익성 = pd.read_csv('./datasets/코스피/코스피수익성.csv',encoding='cp949')
안정성 = pd.read_csv('./datasets/코스피/코스피 안정성.csv',encoding='cp949')
활동성 = pd.read_csv('./datasets/코스피/코스피활동성.csv',encoding='cp949')
성장성 = pd.read_csv('./datasets/코스피/코스피 성장성 계산용.csv',encoding='cp949')
현금흐름 = pd.read_csv('./datasets/코스피/코스피 현금흐름 + 누적수익성비율.csv',encoding='cp949')
연구개발 = pd.read_csv('./datasets/코스피/코스피 rs.csv',encoding='cp949')
EBIT = pd.read_csv('./datasets/코스피/에비~ 코스피.csv',encoding='cp949')
per_pbr_총부채 = pd.read_csv('./datasets/코스피/per pbr 총부채 코스피.csv',encoding='cp949')
직원수_업력 = pd.read_csv('./datasets/코스피/코스피 직원수 급여 업력 외국인 대주주.csv',encoding='cp949')
시가총액 = pd.read_csv('./datasets/코스피/코스피시가총액.csv',index_col=0)  #### 시총도 붙여야됨


In [3]:
df = 안정성.merge(수익성,on=['회사명','거래소코드','회계년도'],how='left')
df = df.merge(활동성,on=['회사명','거래소코드','회계년도'],how='left')
df = df.merge(성장성,on=['회사명','거래소코드','회계년도'],how='left')
df = df.merge(현금흐름,on=['회사명','거래소코드','회계년도'],how='left')
df = df.merge(연구개발,on=['회사명','거래소코드','회계년도'],how='left')
df = df.merge(EBIT,on=['회사명','거래소코드','회계년도'],how='left')
df = df.merge(per_pbr_총부채,on=['회사명','거래소코드','회계년도'],how='left')
df = df.merge(직원수_업력,on=['회사명','거래소코드','회계년도'],how='left')

In [4]:
## 거래소코드 패딩
def padding(x):
    x = str(x)
    return x.rjust(6, "0")
## 외국계기업 제거
def func_isnt_fcompany(x):
    if x[:1] == '9':
        return False
    else:
        return True

# 회계년도컬럼, 년도만 뽑아서 int화
def change_year(x):
    return int(x[:4])

def func_not(x):
    return not x



In [5]:
## 거래소 코드 패딩
df['거래소코드'] = df['거래소코드'].map(padding)
# 회계년도 제거
df['회계년도'] = df['회계년도'].map(change_year)

In [6]:
## 중복되는 데이터 제거
df.drop_duplicates(['거래소코드',"회계년도"], keep='last', inplace=True)
df.reset_index(drop=True, inplace=True)

## 없음


In [7]:
print('전체 행:', df.shape[0],'개')
print('전체 기업 수',len(df['회사명'].unique()))

전체 행: 8010 개
전체 기업 수 797


In [8]:
## 외국계 기업 제거
print("제거할 행 수 : ", len(df[df["거래소코드"].map(func_isnt_fcompany).map(func_not)]))
print('제거할 외국계 기업 수',len(df[~df["거래소코드"].map(func_isnt_fcompany)].회사명.unique()))

df = df[df["거래소코드"].map(func_isnt_fcompany)]
print('남은 데이터 수',df.shape[0]) # 외국계기업 데이터 24개 제거
print('남은 기업 수',len(df['회사명'].unique()))

제거할 행 수 :  24
제거할 외국계 기업 수 5
남은 데이터 수 7986
남은 기업 수 792


In [9]:
## 회사수와 거래소코드수 차이가나는것 확인
print("회사수 : ", len(df["회사명"].unique()))
print("거래소코드수 : ", len(df["거래소코드"].unique()))

회사수 :  792
거래소코드수 :  795


In [13]:
## 거래소 코드 중복기업 전처리
list_overlap_code = list()
for i in list(df["회사명"].unique()):
    if len(df[df["회사명"]==i]["거래소코드"].unique()) > 1:
        list_overlap_code.append(i)
print("거래소코드가 2개인 기업 수", len(list_overlap_code), "개")

거래소코드가 2개인 기업 수 0 개


In [11]:
list_overlap_code

['(주)신성이엔지', '삼성물산(주)', '에스케이(주)']

In [12]:
# 중복 거래소코드 제거
def func_overlap_code(x):
    if x == (df[df["회사명"]==company_name]["거래소코드"].iloc[-1]) :
        return True
    else:
        return False

list_companyname = list(df["회사명"].unique())


dfs = list()


for i in list_companyname:
    company_name = i
    #각 기업별로 데이터프레임 생성
    df_concat = df[df["회사명"]==i]
    df_concat = df_concat[df_concat["거래소코드"].map(func_overlap_code)] # 거래소코드 바뀌기 전의 거래소코드의 데이터는 삭제
    dfs.append(df_concat)
df = pd.concat(dfs)
df.reset_index(drop=True, inplace=True)

In [14]:
## 연결 결측치 개별로 채우기
for i in range(df.shape[0]):
    if pd.isnull(df.loc[i,'유보율(IFRS연결)(백만원)']):
       df.loc[i,'유보율(IFRS연결)(백만원)'] = df.loc[i,'유보율(IFRS)']
    if pd.isnull(df.loc[i,'[제조]* (정상)영업손익(보고서기재)(요약)(IFRS연결)(백만원)']):
       df.loc[i,'[제조]* (정상)영업손익(보고서기재)(요약)(IFRS연결)(백만원)'] = df.loc[i,'[제조]* (정상)영업손익(보고서기재)(요약)(IFRS)(백만원)']
    if pd.isnull(df.loc[i,'[제조]      이익잉여금(결손금)(요약)(IFRS연결)(백만원)']):
       df.loc[i,'[제조]      이익잉여금(결손금)(요약)(IFRS연결)(백만원)'] = df.loc[i,'[제조]      이익잉여금(결손금)(요약)(IFRS)(백만원)']
    if pd.isnull(df.loc[i,'[제조]영업활동으로 인한 현금흐름(간접법)(*)(요약)(IFRS연결)(백만원)']):
       df.loc[i,'[제조]영업활동으로 인한 현금흐름(간접법)(*)(요약)(IFRS연결)(백만원)'] = df.loc[i,'[제조]영업활동으로 인한 현금흐름(간접법)(*)(요약)(IFRS)(백만원)']
    if pd.isnull(df.loc[i,'자산(*)(요약)(IFRS연결)(백만원)']):
       df.loc[i,'자산(*)(요약)(IFRS연결)(백만원)'] = df.loc[i,'자산(*)(요약)(IFRS)(백만원)']
    if pd.isnull(df.loc[i,'부채(*)(요약)(IFRS연결)(백만원)']):
       df.loc[i,'부채(*)(요약)(IFRS연결)(백만원)'] = df.loc[i,'부채(*)(요약)(IFRS)(백만원)']
    if pd.isnull(df.loc[i,'[제조]자본(*)(요약)(IFRS연결)(백만원)']):
       df.loc[i,'[제조]자본(*)(요약)(IFRS연결)(백만원)'] = df.loc[i,'[제조]자본(*)(요약)(IFRS)(백만원)']
    if pd.isnull(df.loc[i,'당기순이익(손실)(요약)(IFRS연결)(백만원)']):
       df.loc[i,'당기순이익(손실)(요약)(IFRS연결)(백만원)'] = df.loc[i,'당기순이익(손실)(요약)(IFRS)(백만원)']
    if pd.isnull(df.loc[i,'법인세비용차감전순이익(손실)(요약)(IFRS연결)(백만원)']):
       df.loc[i,'법인세비용차감전순이익(손실)(요약)(IFRS연결)(백만원)'] = df.loc[i,'법인세비용차감전순이익(손실)(요약)(IFRS)(백만원)']
    if pd.isnull(df.loc[i,'금융원가(요약)(IFRS연결)(백만원)']):
       df.loc[i,'금융원가(요약)(IFRS연결)(백만원)'] = df.loc[i,'금융원가(요약)(IFRS)(백만원)']
    if pd.isnull(df.loc[i,'금융수익(요약)(IFRS연결)(백만원)']):
       df.loc[i,'금융수익(요약)(IFRS연결)(백만원)'] = df.loc[i,'금융수익(요약)(IFRS)(백만원)']
    if pd.isnull(df.loc[i,'매출액(수익)(요약)(IFRS연결)(백만원)']):
       df.loc[i,'매출액(수익)(요약)(IFRS연결)(백만원)'] = df.loc[i,'매출액(수익)(요약)(IFRS)(백만원)']
    if pd.isnull(df.loc[i,'연구개발비(IFRS연결)(천원)']):
       df.loc[i,'연구개발비(IFRS연결)(천원)'] = df.loc[i,'연구개발비(IFRS)(천원)']
    if pd.isnull(df.loc[i,'[제조]      유형자산(요약)(IFRS연결)(백만원)']):
       df.loc[i,'[제조]      유형자산(요약)(IFRS연결)(백만원)'] = df.loc[i,'[제조]      유형자산(요약)(IFRS)(백만원)']
    if pd.isnull(df.loc[i,'자본금회전률(IFRS연결)']):
       df.loc[i,'자본금회전률(IFRS연결)'] = df.loc[i,'자본금회전률(IFRS)']
    if pd.isnull(df.loc[i,'총자본회전률(IFRS연결)']):
       df.loc[i,'총자본회전률(IFRS연결)'] = df.loc[i,'총자본회전률(IFRS)']
    if pd.isnull(df.loc[i,'재고자산회전률(IFRS연결)']):
       df.loc[i,'재고자산회전률(IFRS연결)'] = df.loc[i,'재고자산회전률(IFRS)']
    if pd.isnull(df.loc[i,'유형자산회전율(IFRS연결)']):
       df.loc[i,'유형자산회전율(IFRS연결)'] = df.loc[i,'유형자산회전율(IFRS)']
    if pd.isnull(df.loc[i,'자기자본회전률(IFRS연결)']):
       df.loc[i,'자기자본회전률(IFRS연결)'] = df.loc[i,'자기자본회전률(IFRS)']
    if pd.isnull(df.loc[i,'타인자본회전률(IFRS연결)']):
       df.loc[i,'타인자본회전률(IFRS연결)'] = df.loc[i,'타인자본회전률(IFRS)']
    if pd.isnull(df.loc[i,'경영자본순이익률(IFRS연결)']):
       df.loc[i,'경영자본순이익률(IFRS연결)'] = df.loc[i,'경영자본순이익률(IFRS)']
    if pd.isnull(df.loc[i,'금융손익비율(IFRS연결)']):
       df.loc[i,'금융손익비율(IFRS연결)'] = df.loc[i,'금융비용부담률(IFRS)']
    if pd.isnull(df.loc[i,'총자본순이익률(IFRS연결)']):
       df.loc[i,'총자본순이익률(IFRS연결)'] = df.loc[i,'총자본순이익률(IFRS)']
    if pd.isnull(df.loc[i,'매출액총이익률(IFRS연결)']):
       df.loc[i,'매출액총이익률(IFRS연결)'] = df.loc[i,'매출액총이익률(IFRS)']
    if pd.isnull(df.loc[i,'자기자본순이익률(IFRS연결)']):
       df.loc[i,'자기자본순이익률(IFRS연결)'] = df.loc[i,'자기자본순이익률(IFRS)']
    if pd.isnull(df.loc[i,'총자본정상영업이익률(IFRS연결)']):
       df.loc[i,'총자본정상영업이익률(IFRS연결)'] = df.loc[i,'총자본정상영업이익률(IFRS)']
    if pd.isnull(df.loc[i,'자산(*)(요약)(IFRS연결)']):
       df.loc[i,'자산(*)(요약)(IFRS연결)'] = df.loc[i,'자산(*)(요약)(IFRS)']
    if pd.isnull(df.loc[i,'자본(*)(요약)(IFRS연결)']):
       df.loc[i,'자본(*)(요약)(IFRS연결)'] = df.loc[i,'자본(*)(요약)(IFRS)']
    if pd.isnull(df.loc[i,'현금비율(IFRS연결)(백만원)']):
       df.loc[i,'현금비율(IFRS연결)(백만원)'] = df.loc[i,'현금비율(IFRS)']
    if pd.isnull(df.loc[i,'* (정상)영업손익(보고서기재)(요약)(IFRS연결)(백만원)']):
       df.loc[i,'* (정상)영업손익(보고서기재)(요약)(IFRS연결)(백만원)'] = df.loc[i,'* (정상)영업손익(보고서기재)(요약)(IFRS)(백만원)']
    if pd.isnull(df.loc[i,'차입금의존도(IFRS연결)']):
       df.loc[i,'차입금의존도(IFRS연결)'] = df.loc[i,'차입금의존도(IFRS)(백만원)']
    if pd.isnull(df.loc[i,'감가상각비(IFRS연결)(천원)']):
       df.loc[i,'감가상각비(IFRS연결)(천원)'] = df.loc[i,'감가상각비(IFRS)(천원)']
    if pd.isnull(df.loc[i,'부채비율(IFRS연결)']):
       df.loc[i,'부채비율(IFRS연결)'] = df.loc[i,'부채비율(IFRS)']
    if pd.isnull(df.loc[i,'당좌비율(IFRS연결)']):
       df.loc[i,'당좌비율(IFRS연결)'] = df.loc[i,'당좌비율(IFRS)']
    if pd.isnull(df.loc[i,'*이자비용(요약)(IFRS연결)(백만원)']):
       df.loc[i,'*이자비용(요약)(IFRS연결)(백만원)'] = df.loc[i,'*이자비용(요약)(IFRS)(백만원)']
    if pd.isnull(df.loc[i,'순운전자본비율(IFRS연결)']):
       df.loc[i,'순운전자본비율(IFRS연결)'] = df.loc[i,'순운전자본비율(IFRS)']


In [15]:
drop_col = ['유보율(IFRS)' ,'[제조]* (정상)영업손익(보고서기재)(요약)(IFRS)(백만원)','[제조]      이익잉여금(결손금)(요약)(IFRS)(백만원)',
'[제조]영업활동으로 인한 현금흐름(간접법)(*)(요약)(IFRS)(백만원)','자산(*)(요약)(IFRS)(백만원)', 
'부채(*)(요약)(IFRS)(백만원)','[제조]자본(*)(요약)(IFRS)(백만원)','당기순이익(손실)(요약)(IFRS)(백만원)', 
'법인세비용차감전순이익(손실)(요약)(IFRS)(백만원)','금융원가(요약)(IFRS)(백만원)','금융수익(요약)(IFRS)(백만원)',
'매출액(수익)(요약)(IFRS)(백만원)','연구개발비(IFRS)(천원)',
'[제조]      유형자산(요약)(IFRS)(백만원)','자본금회전률(IFRS)',
'총자본회전률(IFRS)','재고자산회전률(IFRS)','유형자산회전율(IFRS)','자기자본회전률(IFRS)','타인자본회전률(IFRS)',
'경영자본순이익률(IFRS)','금융비용부담률(IFRS)','총자본순이익률(IFRS)','매출액총이익률(IFRS)',
'자기자본순이익률(IFRS)','총자본정상영업이익률(IFRS)','자산(*)(요약)(IFRS)','자본(*)(요약)(IFRS)',
 '현금비율(IFRS)','* (정상)영업손익(보고서기재)(요약)(IFRS)(백만원)',
 '차입금의존도(IFRS)(백만원)','감가상각비(IFRS)(천원)','부채비율(IFRS)',
'당좌비율(IFRS)','*이자비용(요약)(IFRS)(백만원)','순운전자본비율(IFRS)',
'[제조]자산(*)(요약)(IFRS)(백만원)_x','[제조]자산(*)(요약)(IFRS연결)(백만원)_x',
'[제조]자산(*)(요약)(IFRS)(백만원)_y','[제조]자산(*)(요약)(IFRS연결)(백만원)_y',
'[제조]매출액(수익)(요약)(IFRS)(백만원)_y', '[제조]매출액(수익)(요약)(IFRS연결)(백만원)_y',
'[제조]매출액(수익)(요약)(IFRS)(백만원)_x', '[제조]매출액(수익)(요약)(IFRS연결)(백만원)_x']

In [16]:
df.drop(drop_col,axis=1,inplace=True)

In [21]:
df.columns

Index(['회사명', '거래소코드', '회계년도', '자본(*)(요약)(IFRS연결)', '차입금의존도(IFRS연결)',
       '유보율(IFRS연결)(백만원)', '현금비율(IFRS연결)(백만원)', '부채비율(IFRS연결)', '당좌비율(IFRS연결)',
       '순운전자본비율(IFRS연결)', '*이자비용(요약)(IFRS연결)(백만원)',
       '* (정상)영업손익(보고서기재)(요약)(IFRS연결)(백만원)', '감가상각비(IFRS연결)(천원)',
       'EBITDA(IFRS)(백만원)', '총자본정상영업이익률(IFRS연결)', '자기자본순이익률(IFRS연결)',
       '총자본순이익률(IFRS연결)', '매출액총이익률(IFRS연결)', '금융손익비율(IFRS연결)',
       '경영자본순이익률(IFRS연결)', '자본금회전률(IFRS연결)', '유형자산회전율(IFRS연결)',
       '재고자산회전률(IFRS연결)', '자기자본회전률(IFRS연결)', '타인자본회전률(IFRS연결)',
       '총자본회전률(IFRS연결)', '[제조]      이익잉여금(결손금)(요약)(IFRS연결)(백만원)',
       '[제조]영업활동으로 인한 현금흐름(간접법)(*)(요약)(IFRS연결)(백만원)', '연구개발비(IFRS연결)(천원)',
       '금융원가(요약)(IFRS연결)(백만원)', '금융수익(요약)(IFRS연결)(백만원)',
       '법인세비용차감전순이익(손실)(요약)(IFRS연결)(백만원)', '당기순이익(손실)(요약)(IFRS연결)(백만원)',
       '자산(*)(요약)(IFRS연결)(백만원)', '부채(*)(요약)(IFRS연결)(백만원)', '합계_기말인원(명)',
       '합계_년간총급여(천원)', '설립일', '외국인_주식분포비율', '보통주식비율', '누적수익성비율', '유형자산증가율',
       '매출액증가율', '자기자본증가율', '총자본증가율', '정상영업이익증가율'],

In [18]:
## 누적수익성비율
df['누적수익성비율'] = df['[제조]      이익잉여금(결손금)(요약)(IFRS연결)(백만원)']/df['자산(*)(요약)(IFRS연결)']

In [19]:
## 성장성 지표인 증가율 컬럼 만들기
성장성_col = ['[제조]      유형자산(요약)(IFRS연결)(백만원)','매출액(수익)(요약)(IFRS연결)(백만원)',
           '[제조]자본(*)(요약)(IFRS연결)(백만원)','자산(*)(요약)(IFRS연결)',
           '[제조]* (정상)영업손익(보고서기재)(요약)(IFRS연결)(백만원)']

In [20]:
## 증가율 컬럼 전기에 음수일때 당기에 음수지만 증가할때 증가율 계산
## ex (2011년 -10, 2012년 -20) -> -20-(-10) / np.abs(-10)
증가율_df = (df.groupby(['회사명', '거래소코드'])[성장성_col].shift(0) - df.groupby(['회사명', '거래소코드'])[성장성_col].shift(1)) / np.abs(df.groupby(['회사명', '거래소코드'])[성장성_col].shift(1))
증가율_df.rename(columns={'[제조]      유형자산(요약)(IFRS연결)(백만원)':'유형자산증가율', '매출액(수익)(요약)(IFRS연결)(백만원)':'매출액증가율',
       '[제조]자본(*)(요약)(IFRS연결)(백만원)':'자기자본증가율', '자산(*)(요약)(IFRS연결)':'총자본증가율',
       '[제조]* (정상)영업손익(보고서기재)(요약)(IFRS연결)(백만원)':'정상영업이익증가율'},inplace=True)

df.drop(성장성_col,axis=1,inplace=True)
df = pd.concat([df,증가율_df],axis=1)

In [212]:
## 기업별 업력 설립일 이후 지속된 기간
df['업력'] = df['회계년도'] -  pd.to_datetime(df['설립일'],format='%Y/%m/%d').dt.strftime('%Y').astype(int)
df.drop('설립일',axis=1,inplace=True)